<a href="https://colab.research.google.com/github/ArachnidAgrajag/LearnML/blob/master/identify_apparels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#loading the data
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1lrzlULX8VPZQT26hSdtmFWRp2XxFKm0o'})

download.GetContentFile('train_LbELtWX.zip')
!unzip train_LbELtWX.zip

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
# the csv files contain the image file names along with the label
train = pd.read_csv('train.csv')

# we will read all the training images and load them in a list and convert it to
# a numpy array
train_image = []

for i in tqdm(range(train.shape[0])):
  img = image.load_img('train/'+train['id'][i].astype('str')+'.png', 
                         target_size=(28,28,1),
                         color_mode="grayscale")
  # convert image to an numpy array
  img =  image.img_to_array(img)
  # to keep the values from 0 to 1
  img = img/255
  train_image.append(img)

#load the data
X = np.array(train_image)
y = train['label'].values
y = to_categorical(y)

# train test split for validation

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    test_size=0.2)


100%|██████████| 60000/60000 [00:16<00:00, 3580.90it/s]


In [0]:
# define model structure
# We will create a simple architecture (simple lol) with 2 convolutional layers, one dense hidden layer and an output layer.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [5]:
#training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.4966 - accuracy: 0.8243 - val_loss: 0.3093 - val_accuracy: 0.8881
Epoch 2/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.3360 - accuracy: 0.8799 - val_loss: 0.2606 - val_accuracy: 0.9045
Epoch 3/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2855 - accuracy: 0.8969 - val_loss: 0.2449 - val_accuracy: 0.9111
Epoch 4/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2539 - accuracy: 0.9070 - val_loss: 0.2360 - val_accuracy: 0.9139
Epoch 5/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2275 - accuracy: 0.9155 - val_loss: 0.2287 - val_accuracy: 0.9145
Epoch 6/10
48000/48000 [==============================] - 137s 3ms/step - loss: 0.2110 - accuracy: 0.9230 - val_loss: 0.2276 - val_accuracy: 0.9192
Epoch 7/10
48000/48000 [==============================] - 137s

In [13]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# save to gdrive 
model_json_file = drive.CreateFile({'title':'model.json'})
model_json_file.SetContentFile('model.json')
model_json_file.Upload()


# save to gdrive 
model_file = drive.CreateFile({'title':'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()



Saved model to disk


In [0]:
#testing the model
#load test data
download = drive.CreateFile({'id': '1TQ881AHa6HxoyvNd4QDej5vylnqx3Qcd'})
download.GetContentFile('test_ScVgIM0.zip')
!unzip test_ScVgIM0.zip

In [8]:
test = pd.read_csv('test.csv')
test_image = []
for i in tqdm(range(test.shape[0])):
  img = image.load_img('test/'+test['id'][i].astype('str')+'.png', 
                       target_size=(28,28,1), 
                       color_mode='grayscale')
  img = image.img_to_array(img)
  img = img/255
  test_image.append(img)
test = np.array(test_image)
# making predictions

prediction = model.predict_classes(test)

100%|██████████| 10000/10000 [00:02<00:00, 3680.86it/s]


In [0]:


download = drive.CreateFile({'id': '1o99aGmdgTwLqlP3Pi7f6_kBqhjLa5EuK'})
download.GetContentFile('sample_submission_I5njJSF.csv')



In [0]:
# creating submission file
sample = pd.read_csv('sample_submission_I5njJSF.csv')
sample['label'] = prediction
sample.to_csv('sample_cnn.csv', header=True, index=False)


In [0]:
sample_file = drive.CreateFile({'title':'sample_cnn.csv'})
sample_file.SetContentFile('sample_cnn.csv')
sample_file.Upload()